# Initialize the client

In [1]:
import base64
from openai import OpenAI
import os

# Check Image Path

In [2]:
def encode_image(image_path):
    """Encodes a local image to base64."""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Check Audit Image

In [3]:
def audit_image(image_path, audit_type="quality"):
    import jiter
    """
    Audits an image using OpenAI's Vision model.
    
    Args:
        image_path (str): Path to the local image file.
        audit_type (str): Type of audit: 'quality', 'safety', or 'brand'.
    """
    
    # 1. Define the System Prompts based on audit type
    prompts = {
        "quality": """
            You are a Senior QA Specialist for AI Imagery. 
            Audit the image for technical flaws.
            Return a JSON object with these keys:
            - "has_defects": boolean
            - "hand_anatomy_score": 1-10
            - "text_legibility": "clear" or "gibberish"
            - "artifacts_found": [list of specific errors like extra fingers, smudged faces, etc]
        """,
        "safety": """
            You are a Trust & Safety Moderator. 
            Audit this image for NSFW, violence, symbols of hate, or unsafe situations.
            Return a JSON object with:
            - "is_safe": boolean
            - "flagged_elements": [list of items]
            - "content_rating": "G", "PG", "R", or "X"
        """,
        "brand": """
            You are a Brand Compliance Manager.
            Audit this image to ensure it fits our 'Minimalist, Blue & White, Professional' aesthetic.
            Return a JSON object with:
            - "brand_fit_score": 1-10
            - "color_palette_match": boolean
            - "suggested_changes": [list of changes]
        """,
        "creative": """
            You are a Creative Director. Audit this image for 'Visual Storytelling'.
            Return a JSON object with:
            - "narrative_clarity": 1-10 (How quickly is the story understood?)
            - "emotional_hook": "The primary emotion evoked"
            - "composition_critique": "Analysis of balance and framing"
            - "creative_fix": "One bold idea to make this image 10x more impactful"
            - "suggested_changes": [list of changes]
        """,
        "color": """
            You are a Senior Colorist and Accessibility Expert. 
            Analyze the image's color profile and return a JSON object:
            - "dominant_colors": [List of top 3 hex codes or names]
            - "color_harmony": "Complementary", "Analogous", "Triadic", or "Monochromatic"
            - "contrast_ratio": "Pass" or "Fail" (Based on WCAG standards for text over background)
            - "emotional_palette": "Describe the mood (e.g., Somber, Vibrant, Clinical)"
            - "gamut_issues": "Note any oversaturation or 'neon' artifacts common in AI"
            - "suggested_changes": [list of changes]
        """,
        "typography": """
            You are a Lead Type Designer and UX Researcher. 
            Audit the text and typography in this image.
            Return a JSON object with:
            - "detected_text": "The exact transcript of all visible text"
            - "font_classification": "Serif, Sans-Serif, Script, or Display"
            - "visual_hierarchy": "Evaluation of font size contrast between headers and body"
            - "kerning_and_tracking": "Are letter and word spacings consistent or distorted?"
            - "legibility_score": 1-10
            - "font_personality": "The emotional vibe (e.g., Authoritative, Whimsical, Modern)"
            - "technical_errors": [List issues like 'blurred stems', 'irregular counters', or 'hallucinated glyphs']
            - "suggested_changes": [list of changes]
        """,
        "graphic": """
            You are a Senior Production Designer. 
            Audit the layout and graphic elements of this image.
            Return a JSON object with:
            - "composition_type": "Rule of thirds, Centered, Symmetrical, or Asymmetrical"
            - "grid_alignment": "Are elements aligned to a consistent grid or floating randomly?"
            - "visual_weight": "Identify which element has the most 'pull' and if the image feels 'heavy' on one side"
            - "white_space_usage": "Is the design cluttered or breathable? (1-10 score)"
            - "edge_safety": "Are any critical elements too close to the borders (tangents)?"
            - "graphic_artifacts": [List issues like warped shapes, inconsistent line weights, or jagged vectors]
            - "image_resolution": Confirm that resolution falls within the required range of 200–250 DPI.
            - "suggested_changes": [list of changes]
        """,
        "text": """
            You are a Professional Copyeditor and OCR Specialist. 
            Audit the text elements within this image.
            Return a JSON object with:
            - "transcription": "Exact text found in the image"
            - "spelling_grammar": "List any typos or nonsensical AI-generated words"
            - "readability_level": "Grade level (e.g., High School, Professional, Simple)"
            - "tone_consistency": "Does the wording match the visual style? (e.g., Playful vs. Corporate)"
            - "text_placement": "Is the text overlapping vital parts of the image or poorly positioned?"
            - "language_detected": "The primary language used"
            - "suggested_changes": [list of changes]
        """,
        "alignment": """
            You are a Senior Production Artist. 
            Audit the spatial alignment and margins of this image.
            Return a JSON object with:
            - "horizontal_alignment": "Are elements centered, left-aligned, or right-aligned consistently?"
            - "margin_consistency": "Are the 'breathing room' spaces at the top, bottom, and sides equal?"
            - "tangent_check": "Identify if any elements are awkwardly touching the edges or each other."
            - "visual_center_vs_geometric_center": "Does the main subject 'feel' centered, even if it mathematically isn't?"
            - "axis_drift": "Note if text or icons are slightly tilted or off-axis (common AI error)."
            - "recommendation": "One specific instruction to fix the balance (e.g., 'Move text block 20px right')."
            - "margin": Ensure margins fall within 5%–10%.
            - "scaling": Verify vertical and horizontal scaling is set to 100%.
            - "scaling_match": Confirm vertical and horizontal scales are identical.
            - "suggested_changes": [list of changes]
        """
    }

    selected_prompt = prompts.get(audit_type, prompts[audit_type])

    # 2. Encode the image
    base64_image = encode_image(image_path)

    # Initialize the client
    client = OpenAI()

    # 3. Call the API
    try:
        response = client.chat.completions.create(
            model="gpt-4o",  # Use gpt-4o or gpt-4-turbo
            messages=[
                {
                    "role": "system",
                    "content": selected_prompt
                },
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": "Audit this image according to your instructions. Output strictly valid JSON."},
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{base64_image}"
                            },
                        },
                    ],
                }
            ],
            response_format={ "type": "json_object" }, # Forces JSON output
            max_tokens=300,
        )
        return response.choices[0].message.content

    except Exception as e:
        return f"Error: {e}"

# Call Image path 

In [4]:
# --- Usage Example ---
if __name__ == "__main__":
    # Replace with your actual image path
    img_path = "businesscard.jpg" 
    #img_path = input("Enter a Image Path for Audit: ")

    arraudits = ("quality","safety","brand","creative","color","typography","graphic","text","alignment")
    
    # Check if file exists before running
    if os.path.exists(img_path):

        for audit in arraudits:
            print("--- Running "+audit+" Audit ---")
            result = audit_image(img_path, audit_type=audit)
            print(result)
    else:
        print(f"Please create a dummy file named '{img_path}' to test the script.")

--- Running quality Audit ---
{
    "has_defects": false,
    "hand_anatomy_score": null,
    "text_legibility": "clear",
    "artifacts_found": []
}
--- Running safety Audit ---
{"is_safe": true, "flagged_elements": [], "content_rating": "G"}
--- Running brand Audit ---
{
    "brand_fit_score": 9,
    "color_palette_match": true,
    "suggested_changes": [
        "Consider adding a touch more white space for a cleaner look.",
        "Slightly reduce the size of the contact information icons to enhance minimalism."
    ]
}
--- Running creative Audit ---
{
    "narrative_clarity": 8,
    "emotional_hook": "Professionalism",
    "composition_critique": "The design is clean and well-organized, with a good balance between text and the image. The use of blue adds a sense of trust and stability. The circular photo draws attention and complements the wavy lines extending from it.",
    "creative_fix": "Add a subtle background pattern or texture to increase depth and interest without overwhe